In [2]:
import sys
!{sys.executable} -m pip install praw

     |████████████████████████████████| 152 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 15.3 MB/s eta 0:00:01


In [4]:
import datetime

import praw
import pandas as pd

from keys import client_id, client_secret


In [5]:
reddit = praw.Reddit( client_id=client_id,
                      client_secret=client_secret,
                      user_agent='android:my_app:v1 (by /u/Agreeable_Sky4093)')

In [6]:
chelsea = reddit.subreddit('ChelseaFC')

#Gathering the top 500 posts, with their title, url, body, upvotes, timestamp, and an index that serves as a key between the
#posts and the comments we collect later
posts = []
for index, post in enumerate(chelsea.top(limit=500)):
    posts.append([post.title, "https://www.reddit.com" + post.permalink, post.selftext, post.score, post.created_utc, index])

#Converting into DataFrame
posts = pd.DataFrame(posts, columns=['Title', 'URL', 'Body', 'Upvotes', 'Time', 'Key'])
#Changing from utc time to standard timestamp
posts.Time = posts.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

#The first post is a sticky, so we can drop it
posts = posts.iloc[1:]

In [7]:
posts.head(3)

,Title,URL,Body,Upvotes,Time,Key
1,"Thank you, Eden.",https://www.reddit.com/r/chelseafc/comments/bu...,,6157,2019-05-30 08:57:30,1
2,Courtois now having his medical at Madrid!,https://www.reddit.com/r/chelseafc/comments/95...,,5975,2018-08-08 22:33:45,2
3,This man has gone toe to toe with the very bes...,https://www.reddit.com/r/chelseafc/comments/hf...,,5526,2020-06-26 09:33:52,3


In [8]:
posts.shape

(499, 6)

In [9]:
def collect_replies(key, url):
    ''' 
    params pandas series row: each row of the dataframe we built above in the form of a panda series
    Returns a pandas DataFrame, where each row represents an individual comment
    '''
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:] 

    table = {'Reply':[], 'Upvote':[], 'Time':[], 'Key':[]}

    while comment_queue:
        comment = comment_queue.pop(0)
        table['Reply'].append(comment.body)
        table['Time'].append(comment.created_utc)
        table['Upvote'].append(comment.score)
        table['Key'].append(key)
        comment_queue.extend(comment.replies)
    
    return pd.DataFrame.from_dict(table)

In [10]:
#Let us first generate a list of tupules that contains the key and url for each row - the first value of the tupule is key,
#and the second value is url
keys = posts.Key.tolist()
urls = posts.URL.tolist()
tupules = list(zip(keys, urls))

#Now we generate our comments dataframe using list comprehensions!
comments = pd.concat([collect_replies(x[0], x[1]) for x in tupules])

In [11]:
#Again, converting the timestamp from utc to a standard format
comments.Time = comments.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

In [12]:
comments.shape


(48699, 4)

In [15]:
#comments.to_csv('Comments_.csv', index=False)

In [16]:
#posts.to_csv('Posts.csv', index=False)